In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
import torch

In [50]:
class Wav2vec(nn.Module):

    def __init__(self):
        super(Wav2vec, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        activation = nn.ReLU()
        dropout = 0.0
        self.encoder = Encoder()

    def forward(self, x):
        z = self.encoder(x)
        #c = self.context(x)
        # x = x.view(-1, self.num_flat_features(x))
        return z#, c
    
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        dropout=0.0
        self.in_c = 10
        activation = nn.ReLU()
        self.encoder = nn.Sequential(nn.Conv1d(in_channels=1, out_channels=self.in_c, kernel_size=10, stride=5),
                                     nn.Dropout(p=dropout),
                                     nn.GroupNorm(1, self.in_c),  # Affine, what to do?
                                     activation,
                                     # 2nd layer
                                     nn.Conv1d(in_channels=self.in_c, out_channels=self.in_c, kernel_size=8, stride=4),
                                     nn.Dropout(p=dropout),
                                     ## See norm_block - FB_repo
                                     nn.GroupNorm(1, self.in_c),  # Affine, what to do?
                                     activation,
                                     # 3rd layer
                                     nn.Conv1d(in_channels=self.in_c, out_channels=self.in_c, kernel_size=4, stride=2),
                                     nn.Dropout(p=dropout),
                                     nn.GroupNorm(1, self.in_c),  # Affine, what to do?
                                     activation,
                                     # Fourth layer
                                     nn.Conv1d(in_channels=self.in_c, out_channels=self.in_c, kernel_size=4, stride=2),
                                     nn.Dropout(p=dropout),
                                     nn.GroupNorm(1, self.in_c),  # Affine, what to do?
                                     activation,
                                     # Fifth layer
                                     nn.Conv1d(in_channels=self.in_c, out_channels=self.in_c, kernel_size=4, stride=2),
                                     nn.Dropout(p=dropout),
                                     nn.GroupNorm(1, self.in_c),  # Affine, what to do?
                                     activation)
    def log_compression(self, x):
        # https://www.edn.com/log-1-x-compression/
        x = x.abs()
        x = x + 1
        return x.log()

    def forward(self, x):
        x = self.encoder(x)
        x = self.log_compression(x)
        # TODO implement skipped connections?
        return x

In [51]:
waveform, sample_rate = torchaudio.load("wav_16k_example.wav")

In [52]:
torch.unsqueeze(waveform, 1).shape

torch.Size([1, 1, 31440])

In [53]:
m = Wav2vec()
m(waveform.unsqueeze(1))

tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.6562, 0.6167, 0.6604,  ..., 0.6695, 0.6726, 0.6344],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.8062, 0.8101, 0.8068,  ..., 0.8020, 0.7977, 0.8037],
         [0.1249, 0.1496, 0.1190,  ..., 0.1110, 0.1435, 0.1550],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],
       grad_fn=<LogBackward>)